In [ ]:
#default_exp ranker

In [ ]:
#hide
from nbdev.showdoc import *

# Ranker

Takes a query and an index and finds the nearest neighbors or most similar scores. Ideally this is just a simple Annoy `get_nns_by_vector`, or in the simple case a similarity score across all the vectors.

In [ ]:
import torch


from pathlib import Path

from memery.loader import treemap_loader, db_loader
from memery.encoder import text_encoder

/home/mage/.local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
treemap = treemap_loader(Path('./images/memery.ann'))

In [ ]:
if treemap:
    treemap.get_n_items()

In [ ]:
#export
def ranker(query_vec, treemap):
    nn_indexes = treemap.get_nns_by_vector(query_vec[0], treemap.get_n_items())
    return(nn_indexes)

In [ ]:
#export
def nns_to_files(db, indexes):
#     return([[v['fpath'] for k,v in db.items() if v['index'] == ind][0] for ind in indexes])
    return([db[ind]['fpath'] for ind in indexes])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
db = db_loader(Path('images/memery.pt'), device)

In [ ]:
query = 'dog'

In [ ]:
query_vec = text_encoder(query, device)
indexes = ranker(query_vec, treemap)
ranked_files = nns_to_files(db, indexes)

In [ ]:
ranked_files[:5]

['images/Wholesome-Meme-8.jpg',
 'images/Wholesome-Meme-5.jpg',
 'images/Wholesome-Meme-35.jpg',
 'images/Wholesome-Meme-67.png',
 'images/embarassed-dog-on-bed-SA2BDZW.jpg']